In [1]:
import os
import sqlite3
import hashlib
from typing import List, Tuple, Optional
import jinja2
from jinja2 import meta

from dotenv import load_dotenv
load_dotenv()

True

# Database

In [2]:
connection = sqlite3.connect(os.getenv('DB_PATH'))

# Create a cursor object.
cursor = connection.cursor()

In [ ]:
cursor.execute("""
                CREATE TABLE IF NOT EXISTS projects (
                    id TEXT PRIMARY KEY,
                    name TEXT NOT NULL,
                    description TEXT,
                    created_at TEXT NOT NULL,
                    updated_at TEXT NOT NULL
                )""")


In [ ]:
cursor.execute("""
                CREATE TABLE IF NOT EXISTS prompts (
                    id TEXT PRIMARY KEY,
                    prompt_group_id TEXT NOT NULL,
                    project_id TEXT NOT NULL,
                    parent_prompt_id TEXT,
                    name TEXT NOT NULL,
                    description TEXT,
                    version INTEGER NOT NULL,
                    prompt_template TEXT NOT NULL,
                    input_variables TEXT,
                    favourite INTEGER NOT NULL,
                    notes TEXT,
                    created_at TEXT NOT NULL,
                    updated_at TEXT NOT NULL,
                    FOREIGN KEY (project_id) REFERENCES projects (id)
                )""")

In [ ]:
cursor.execute("""
                CREATE TABLE IF NOT EXISTS temp_prompt_bucket (
                    
                )""")

In [11]:
cursor.execute("""
UPDATE prompts
SET
    parent_prompt_id = "30488bc93ea34cebb8ccee107d44b354b517fb6e40cde1146e2c1fa3024f62a6,2b57ee0ac06f1379a12140bb4545692ee047a21bcc30753f204daa59a3cb7396"
WHERE
    id = "74d3eaacbae7f2327f8b0ec902766086450336680d7e0f0fec407cb0b4812c97"
""")
connection.commit()

In [ ]:
# Add New Project
def create_project(project_name:str, connection:sqlite3.Connection, cursor:sqlite3.Cursor, description=None):
    # Check if project name already exists (case-insensitive)
    cursor.execute("SELECT COUNT(*) FROM projects WHERE LOWER(name) = LOWER(?)", (project_name,))
    count = cursor.fetchone()[0]
    if count > 0:
        raise ValueError("Project name already exists")

    # Generate unique ID using hashlib's sha
    project_id = hashlib.sha256(project_name.encode()).hexdigest()

    # Insert new record into the project table
    if description is None:
        cursor.execute("INSERT INTO projects (id, name) VALUES (?, ?)", (project_id, project_name))
    else:
        cursor.execute("INSERT INTO projects (id, name, description) VALUES (?, ?, ?)", (project_id, project_name, description))
    connection.commit()


In [ ]:
def update_project_name(project_id: str, new_name: str, connection: sqlite3.Connection, cursor: sqlite3.Cursor):
    # Check if project ID exists
    cursor.execute("SELECT COUNT(*) FROM projects WHERE id = ?", (project_id,))
    count = cursor.fetchone()[0]
    if count == 0:
        raise ValueError("Project ID does not exist")

    # Check if new name already exists (case-insensitive)
    cursor.execute("SELECT COUNT(*) FROM projects WHERE LOWER(name) = LOWER(?) AND id != ?", (new_name, project_id))
    count = cursor.fetchone()[0]
    if count > 0:
        raise ValueError("New name already exists")

    # Update the project name
    cursor.execute("UPDATE projects SET name = ? WHERE id = ?", (new_name, project_id))
    connection.commit()


In [ ]:
def get_project_id(project_name: str, connection: sqlite3.Connection, cursor: sqlite3.Cursor) -> str:
    # Retrieve the project ID based on the project name (case-insensitive)
    cursor.execute("SELECT id FROM projects WHERE LOWER(name) = LOWER(?)", (project_name,))
    result = cursor.fetchone()
    if result is None:
        raise ValueError("Project name not found")
    return result[0]


In [ ]:
def get_all_projects(cursor: sqlite3.Cursor) -> List[Tuple[str, str, Optional[str]]]:
    cursor.execute("SELECT * FROM projects")
    projects = cursor.fetchall()
    return projects


In [ ]:
get_all_projects(cursor)

In [ ]:
create_project("Test Project", connection, cursor, "This is a test project")

In [ ]:
get_project_id("Test Project", connection, cursor)

In [ ]:
project_id = get_project_id("Test Project", connection, cursor)
update_project_name(project_id, "Test Project 1", connection, cursor)

In [ ]:
def update_project_description(project_id: str, description: str, connection: sqlite3.Connection, cursor: sqlite3.Cursor):
    # Check if project ID exists
    cursor.execute("SELECT COUNT(*) FROM projects WHERE id = ?", (project_id,))
    count = cursor.fetchone()[0]
    if count == 0:
        raise ValueError("Project ID does not exist")

    # Update the project description
    cursor.execute("UPDATE projects SET description = ? WHERE id = ?", (description, project_id))
    connection.commit()


In [ ]:
update_project_description(project_id, "This is a test project 1", connection, cursor)

In [ ]:
def add_prompt_template(
    project_id: str,
    name: str,
    version: int,
    prompt_template: str,
    input_variables: str,
    favourite: bool,
    notes: str,
    connection: sqlite3.Connection,
    cursor: sqlite3.Cursor
):
    # Check if project ID exists
    cursor.execute("SELECT COUNT(*) FROM projects WHERE id = ?", (project_id,))
    count = cursor.fetchone()[0]
    if count == 0:
        raise ValueError("Project ID does not exist")

    # Check if prompt name already exists within the project (case-insensitive)
    cursor.execute("SELECT COUNT(*) FROM prompts WHERE LOWER(name) = LOWER(?) AND project_id = ?", (name, project_id))
    count = cursor.fetchone()[0]
    if count > 0:
        raise ValueError("Prompt name already exists within the project")

    # Generate unique ID using hashlib's sha
    prompt_id = hashlib.sha256((project_id + name + str(version)).encode()).hexdigest()

    # Insert new record into the prompts table
    cursor.execute("""
                    INSERT INTO prompts (id, project_id, name, version, prompt_template, input_variables, favourite, notes)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    """, (prompt_id, project_id, name, version, prompt_template, input_variables, int(favourite), notes))
    connection.commit()


In [ ]:
prompt = """Generate {{k}} questions that test reader comprehension of the following text.

Text: {{text}}

Questions:"""
add_prompt_template(project_id, "Test Prompt", 1, prompt, "k, text", False, "This is a test note", connection, cursor)

In [3]:
def update_prompt_template(
    prompt_id: str,
    new_prompt_template: str,
    new_input_variables: str,
    new_favourite: bool,
    new_notes: str,
    connection: sqlite3.Connection,
    cursor: sqlite3.Cursor
    ):
    # Check if prompt ID exists
    cursor.execute("SELECT COUNT(*) FROM prompts WHERE id = ?", (prompt_id,))
    count = cursor.fetchone()[0]
    if count == 0:
        raise ValueError("Prompt ID does not exist")

    # Update the prompt template, input variables, favourite, and notes
    cursor.execute("""
                    UPDATE prompts
                    SET prompt_template = ?, input_variables = ?, favourite = ?, notes = ?
                    WHERE id = ?
                    """, (new_prompt_template, new_input_variables, int(new_favourite), new_notes, prompt_id))
    connection.commit()


In [ ]:
def get_prompt_id_by_name_and_version(project_id: str, name: str, version: int, connection: sqlite3.Connection, cursor: sqlite3.Cursor) -> str:
    # Retrieve the prompt ID based on the project ID, name, and version
    cursor.execute("SELECT id FROM prompts WHERE project_id = ? AND LOWER(name) = LOWER(?) AND version = ?", (project_id, name, version))
    result = cursor.fetchone()
    if result is None:
        raise ValueError("Prompt not found")
    return result[0]

In [ ]:
def get_all_prompts(connection: sqlite3.Connection, cursor: sqlite3.Cursor, project_id: Optional[str]=None) -> List[Tuple[str, str, Optional[str]]]:
    # Retrieve all prompts for the given project ID
    if project_id:
        cursor.execute("SELECT * FROM prompts WHERE project_id = ?", (project_id,))
    else:
        cursor.execute("SELECT * FROM prompts")
    prompts = cursor.fetchall()
    return prompts


In [ ]:
project_id = get_project_id("Test Project", connection, cursor)

In [ ]:
get_all_prompts(project_id, connection, cursor)

In [ ]:
get_prompt_id_by_name_and_version(project_id, "Test Prompt", 1, connection, cursor)


In [ ]:
prompt_id = get_prompt_id_by_name_and_version(project_id, "Test Prompt", 1, connection, cursor)
prompt = """Generate {{ k }} questions that test reader comprehension of the following text.

Text: {{ text }}

Questions:"""

update_prompt_template(prompt_id, prompt, "k, text", True, "This is a test note 1", connection, cursor)

In [ ]:
def delete_prompt(id:str, connection:sqlite3.Connection, cursor:sqlite3.Cursor):
    cursor.execute("DELETE FROM prompts WHERE id = ?", (id,))
    connection.commit()

In [ ]:
delete_prompt("77bf29bb34442ac8ada6c9c07e760b500004ac7e43cf8642ff731f54a35a900f", connection, cursor)

# Templates

In [25]:
env = jinja2.Environment()
    
# Get the Prompt Template
prompt_template = env.from_string("Here is a template. And has input variables: {{input_variables}} and some more text - {{text}}")
prompt_template.render()

'Here is a template. And has input variables:  and some more text - '

In [26]:
# Find variable names
meta.find_undeclared_variables(env.parse("Here is a template. And has input variables: {{input_variables}} and some more text - {{text}}"))

{'input_variables', 'text'}

# Next Section Here